# Arbres de décision 

decision trees


## Application à un vrai jeu de données

Peut-on prédire si un client va acheter une assurance ou pas?

[dataset Caravan](https://islp.readthedocs.io/en/latest/datasets/Caravan.html)


Nous allons utiliser un [Classifieur par arbre de décision](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) et comparer les résultats avec kNN.

Le but est d'avoir au final l'algorithme le plus performant pour répondre à notre problématique.

### Classification de référence (baseline)

In [1]:
from ISLP import load_data
from ISLP import confusion_table
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 

Utiliser un [Classifieur par arbre de décision](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) en laissant les paramètres par défaut de sklearn.

Afficher la table de confusion pour l'évaluation du jeu de test.

In [2]:
caravan = load_data('Caravan')

y = caravan.Purchase.map(lambda x: 1 if x=='Yes' else 0)  # Pour les métriques de scikit learn
X = caravan.drop(columns=['Purchase'])

(X_train,
 X_test,
 y_train,
 y_test) = train_test_split(X,
                            y,
                            test_size=1164,
                            random_state=0)



classifier = DecisionTreeClassifier(random_state=42)
classifier.fit(X_train, y_train)

y_test_pred = classifier.predict(X_test)
confusion_table(y_test_pred, y_test)


Truth,0,1
Predicted,,
0,1015,64
1,72,13


Calculer le score de notre métrique: tp/(tp+fp)

In [3]:
13/(13+72)

0.15294117647058825

15,3%: pas encore mieux que knn où l'on atteignait 15,8%

Quel est l'effet de la normalisation avant l'entraînement d'un arbre de décision?

La normalisation n'est pas utile pour les arbres de décision

Définition d'une fonction pour mesurer notre métrique:

In [4]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_test_pred)  # ! inversé par rapport à la fonction de ISLP
cm # ! affichage transposé par rapport à celui de ISLP

array([[1015,   72],
       [  64,   13]])

In [5]:
fp = cm[0][1]
tp = cm[1][1]

tp/(tp+fp)

0.15294117647058825

Définition d'une fonction pour mesurer notre métrique:

In [6]:
from sklearn.metrics import confusion_matrix

# La valeur prédictive positive : Positive predictive value (PPV), precision
def metric(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    fp = cm[0][1]
    tp = cm[1][1]
    
    return tp/(tp+fp)

In [7]:
metric(y_test, y_test_pred)

0.15294117647058825

Métrique classique appelée la valeur prédictive positive (en anglais: `Positive predictive value (PPV), precision`)

In [8]:
from sklearn.metrics import precision_score

precision_score(y_test, y_test_pred)

0.15294117647058825

### Exploration des hyperparamètres

On peut ensuite faire une exploration des hyperparamètres pour essayer d'améliorer ce résultat.

Paramètres de la fonction [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) de scikit-learn à explorer (a minima):
- criterion
- max_depth

Quelle est la meilleur combinaison d'hyperparamètres?

In [9]:
criterions =['gini', 'entropy', 'log_loss']
max_depths = [3, 5, 10, 15, 20, None]

results = []
for criterion in criterions:
    for max_depth in max_depths:
        classifier = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, random_state=42)
        classifier.fit(X_train, y_train)
        
        y_test_pred = classifier.predict(X_test)
        results.append((precision_score(y_test, y_test_pred), criterion, max_depth))

sorted(results)

[(0.08, 'entropy', 10),
 (0.08, 'log_loss', 10),
 (0.15294117647058825, 'gini', None),
 (0.16216216216216217, 'entropy', None),
 (0.16216216216216217, 'log_loss', None),
 (0.1694915254237288, 'gini', 15),
 (0.17142857142857143, 'entropy', 20),
 (0.17142857142857143, 'log_loss', 20),
 (0.17307692307692307, 'entropy', 15),
 (0.17307692307692307, 'log_loss', 15),
 (0.17647058823529413, 'entropy', 5),
 (0.17647058823529413, 'log_loss', 5),
 (0.18181818181818182, 'gini', 20),
 (0.22727272727272727, 'gini', 10),
 (0.2857142857142857, 'gini', 5),
 (0.3333333333333333, 'entropy', 3),
 (0.3333333333333333, 'log_loss', 3),
 (0.375, 'gini', 3)]

In [10]:
classifier = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=42)
classifier.fit(X_train, y_train)

y_test_pred = classifier.predict(X_test)
confusion_table(y_test_pred, y_test)

Truth,0,1
Predicted,,
0,1085,76
1,2,1


On a certes un bon score, mais très peu de valeurs sélectionnées. Pour réussir à vendre nos contrats d'assurance, c'est pas terrible.

Si on veut que nos commerciaux aient un nombre suffisant de prospects à appeler il faudrait contraindre notre solution a choisir un algorithme qui renvoit un nombre suffisant de résultats prédits positifs.

Disons au minium `19`, ce qu'on avait avec knn.

Quelle est dans ce cas la meilleur combinaison d'hyperparamètres?

In [11]:
criterions =['gini', 'entropy', 'log_loss']
max_depths = [3, 5, 10, 15, 20, None]

results = []
for criterion in criterions:
    for max_depth in max_depths:
        classifier = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, random_state=42)
        classifier.fit(X_train, y_train)
        
        y_test_pred = classifier.predict(X_test)
        if sum(y_test_pred) >= 19:
            results.append((precision_score(y_test, y_test_pred), criterion, max_depth))

sorted(results)

[(0.08, 'entropy', 10),
 (0.08, 'log_loss', 10),
 (0.15294117647058825, 'gini', None),
 (0.16216216216216217, 'entropy', None),
 (0.16216216216216217, 'log_loss', None),
 (0.1694915254237288, 'gini', 15),
 (0.17142857142857143, 'entropy', 20),
 (0.17142857142857143, 'log_loss', 20),
 (0.17307692307692307, 'entropy', 15),
 (0.17307692307692307, 'log_loss', 15),
 (0.18181818181818182, 'gini', 20),
 (0.22727272727272727, 'gini', 10)]

### Meilleur résultat

In [12]:
classifier = DecisionTreeClassifier(criterion='gini', max_depth=10, random_state=42)
classifier.fit(X_train, y_train)

y_test_pred = classifier.predict(X_test)
confusion_table(y_test_pred, y_test)

Truth,0,1
Predicted,,
0,1053,67
1,34,10


In [13]:
precision_score(y_test, y_test_pred)

0.22727272727272727

### Affinons la solution

Est-il possible d'améliorer le résultat obtenu?

In [14]:
criterion='gini'
max_depths = list(range(7,15))

results = []
for max_depth in max_depths:
    classifier = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, random_state=42)
    classifier.fit(X_train, y_train)
    
    y_test_pred = classifier.predict(X_test)
    if sum(y_test_pred) >= 19:
        results.append((precision_score(y_test, y_test_pred), criterion, max_depth))

sorted(results)

[(0.2222222222222222, 'gini', 11),
 (0.2222222222222222, 'gini', 14),
 (0.22448979591836735, 'gini', 12),
 (0.22727272727272727, 'gini', 10),
 (0.24528301886792453, 'gini', 13),
 (0.25, 'gini', 7),
 (0.25, 'gini', 9),
 (0.2727272727272727, 'gini', 8)]

In [15]:
classifier = DecisionTreeClassifier(criterion='gini', max_depth=8, random_state=42)
classifier.fit(X_train, y_train)

y_test_pred = classifier.predict(X_test)
print('valeur prédictive positive: {0:0.0f} %'.format(precision_score(y_test, y_test_pred)*100))
confusion_table(y_test_pred, y_test)

valeur prédictive positive: 27 %


Truth,0,1
Predicted,,
0,1071,71
1,16,6
